In [1]:
import pandas as pd
import numpy as np

from data.source import AwsClient, Position, Extract
from data.reshape import Transform

# Config
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
aws = AwsClient()

In [3]:
aws.load_data_s3()

WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (71838) not 512 + multiple of sector size (512)
WARNING *** file size (71834) not 512 + multiple of sector size (512)
WARNING *** file size (69788) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (60570) not 512 + multiple of sector size (512)
WARNING *** file size (60572) not 512 + multiple of sector size (512)
WARNING *** file size (59548) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59548) not 512 + multiple of sector size (512)
WARNING *** file size (59546) not 512 + multiple of sector size (512)
WARNING *** file size (59548) not 512 + multiple of sector size (512)
WARNING *** file siz

In [4]:
position = Position(aws.df_list_pos)

## Position dataset

In [5]:
extract = Extract(2010, 2021, aws.extrato, aws.extrato_bolsa)

/Users/gabriellopes/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/gabriellopes/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1675: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [6]:
extract.extract_fis.head()

,Nome,ano,mes,Data,Vlr Aporte,Vlr Resgate,Vlr IR,Rendimento Resgatado,Tipo,Data2
0,AZ Quest Multi FIC FIM,2017,3,2017-03-31,"15,000.00",0.00,0.00,0.00,FI,2017-03-31
1,Azul Quantitativo FIM,2013,6,2013-06-30,"5,000.00",0.00,0.00,0.00,FI,2013-06-30
2,Azul Quantitativo FIM,2013,8,2013-08-31,"10,200.00",0.00,0.00,0.00,FI,2013-08-31
3,BNP Paribas Inflação FI RF,2015,1,2015-01-31,"5,000.00",0.00,0.00,0.00,FI,2015-01-31
4,BNP Paribas Inflação FI RF,2015,2,2015-02-28,"5,300.00",0.00,0.00,0.00,FI,2015-02-28


In [7]:
transform = Transform()

df = transform.resume(position, extract)

/Users/gabriellopes/Documents/Pessoal/dev/new-finance-app/data/reshape.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Tipo'] = np.where(df1['Papel'].str.contains('34'),
/Users/gabriellopes/Documents/Pessoal/dev/new-finance-app/data/reshape.py:169: RuntimeWarning: divide by zero encountered in double_scalars
  rend_per_ = valor / df_[df_.index == row - 1]['Financeiro'].sum() * 100
/Users/gabriellopes/Documents/Pessoal/dev/new-finance-app/data/reshape.py:158: RuntimeWarning: divide by zero encountered in double_scalars
  rend_per_ = rend_ / df_[df_.index <= row]['aporte'].sum() * 100
/Users/gabriellopes/Documents/Pessoal/dev/new-finance-app/data/reshape.py:158: RuntimeWarning: divide by zero encountered in double_scalars
  rend_per_ = re

In [8]:
extract.extract_fis['Data2'].unique()

array(['2017-03-31T00:00:00.000000000', '2013-06-30T00:00:00.000000000',
       '2013-08-31T00:00:00.000000000', '2015-01-31T00:00:00.000000000',
       '2015-02-28T00:00:00.000000000', '2017-10-31T00:00:00.000000000',
       '2019-09-30T00:00:00.000000000', '2015-09-30T00:00:00.000000000',
       '2020-07-31T00:00:00.000000000', '2020-08-31T00:00:00.000000000',
       '2012-01-31T00:00:00.000000000', '2016-11-30T00:00:00.000000000',
       '2016-04-30T00:00:00.000000000', '2014-01-31T00:00:00.000000000',
       '2018-12-31T00:00:00.000000000', '2013-01-31T00:00:00.000000000',
       '2013-05-31T00:00:00.000000000', '2010-10-31T00:00:00.000000000',
       '2010-11-30T00:00:00.000000000', '2018-04-30T00:00:00.000000000',
       '2018-06-30T00:00:00.000000000', '2020-03-31T00:00:00.000000000',
       '2021-01-31T00:00:00.000000000', '2014-04-30T00:00:00.000000000',
       '2015-11-30T00:00:00.000000000', '2020-10-31T00:00:00.000000000',
       '2012-05-31T00:00:00.000000000', '2016-03-31

In [9]:
# Estou aqui!!! 
# Colocar o metodo na classe utils e aplicar no extract.extract_fis.

import datetime 
any_date = datetime.datetime(2020, 9, 23)

def last_day_of_month(date):
    # Guaranteed to get the next month. Force any_date to 28th and then add 4 days.
    next_month = date.replace(day=28) + datetime.timedelta(days=4)
    
    # Subtract all days that are over since the start of the month.
    day = next_month - datetime.timedelta(days=next_month.day)
    return day

last_day_of_month(any_date)

extract.extract_fis.head()['Data'].apply(lambda x: last_day_of_month(x))

0   2017-03-31
1   2013-06-30
2   2013-08-31
3   2015-01-31
4   2015-02-28
Name: Data, dtype: datetime64[ns]

In [10]:
df.head()

,level_0,index,Nome,Qtd Disponivel,Cotacao,Financeiro,period,mes,ano,Data,Tipo,dividendo,aporte,retirada,rendimento_resgatado,rendimento,%,periodo_cont
0,5,5.00,BPAC11,200.00,33.24,"6,648.00",2020/3,3,2020,2020-03-31,Ação,0.00,"7,190.00",0.00,0.00,-542.00,-7.54,1
1,0,0.00,BPAC11,200.00,42.30,"8,460.00",2020/4,4,2020,2020-04-30,Ação,0.00,0.00,0.00,0.00,"1,812.00",27.26,2
2,43,43.00,BPAC11,200.00,48.84,"9,768.00",2020/5,5,2020,2020-05-29,Ação,0.00,0.00,0.00,0.00,"1,308.00",15.46,3
3,53,53.00,BPAC11,78.00,49.60,"1,934.40",2020/6,6,2020,2020-06-30,Ação,0.00,0.00,"15,500.00",0.00,"7,666.40",78.48,4
4,12,12.00,BPAC11,26.00,86.42,"2,246.92",2020/7,7,2020,2020-07-31,Ação,0.00,0.00,0.00,0.00,312.52,16.16,5


In [11]:
df[df['Data'] == 0]

,level_0,index,Nome,Qtd Disponivel,Cotacao,Financeiro,period,mes,ano,Data,Tipo,dividendo,aporte,retirada,rendimento_resgatado,rendimento,%,periodo_cont


In [12]:
df_extract = transform.transform_extract_stocks(extract)

In [13]:
position.fis[
    (position.fis['Nome'] == 'AZ Quest Multi FIC FIM') &
    #(position.fis['mes'] == 3) &
    (position.fis['ano'] == 2020)
].head()

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period,mes,ano,data_posicao
20,AZ Quest Multi FIC FIM,2020-02-28,"1,464.80",12.95,"18,967.40",93.33,0.00,"18,874.07",0,"18,967.40",2020/2,2,2020,2020-02-28
219,AZ Quest Multi FIC FIM,2020-01-31,"1,464.80",12.84,"18,801.04",68.38,0.00,"18,732.66",0,"18,801.04",2020/1,1,2020,2020-01-31


In [14]:
df_return = pd.DataFrame([])

for ativo in df['Papel'].unique():
    df_ = df[df['Papel'] == ativo].sort_values(by=['ano', 'mes']).reset_index()
    rendimento = []
    rendimento_percent = []
    periodo_cont = []
    count = 0

    for row in df_.index:
        if (df_[df_.index == row]['Financeiro'].sum() > 1) | (df_[df_.index == row]['retirada'].sum() > 1):
            if count == 0: # primeiro rendimento
                count += 1
                rend_ = df_[df_.index == row]['Financeiro'].sum() - df_[df_.index <= row]['aporte'].sum()
                rend_per_ = rend_ / df_[df_.index <= row]['aporte'].sum() * 100

                rendimento.append(rend_)
                rendimento_percent.append(rend_per_)
                periodo_cont.append(1)
            else:
                count += 1
                valor = df_[df_.index == row]['Financeiro'].sum()\
                            + df_[df_.index == row]['retirada'].sum()\
                            - df_[df_.index == row -1]['Financeiro'].sum()\
                            - df_[df_.index == row]['aporte'].sum()
                rend_per_ = valor / df_[df_.index == row - 1]['Financeiro'].sum() * 100
                rendimento.append(valor)
                rendimento_percent.append(rend_per_)
                periodo_cont.append(count)
                
        else:
            rendimento.append(0)
            rendimento_percent.append(0)
            periodo_cont.append(0)
            count = 0

    df_['rendimento'] = rendimento
    df_['rendimento_percent'] = rendimento_percent
    df_['periodo_cont'] = periodo_cont
    #df_['rendimento_acum'] = df_['rendimento'].cumsum()
    df_return = df_return.append(df_)

KeyError: 'Papel'

In [8]:
df2 = extract.extract_fis.rename(columns={
    'Nome': 'Papel'#, 
    #'Vlr Aporte': 'aporte', 
    #'Vlr Resgate': 'retirada'
})

df2 = df2.groupby(['Papel', 'ano', 'mes'])\
        .agg(vlr_aporte=('Vlr Aporte', 'sum'), 
            vlr_resgate=('Vlr Resgate', 'sum'), 
            rendimento_resgatado=('Rendimento Resgatado', 'sum')).reset_index()

df3 = df.merge(df2, 
        how='outer', 
        left_on=['ano', 'mes', 'Papel'], 
        right_on=['ano', 'mes', 'Papel']).fillna(0)

df3['aporte'] = df3['vlr_aporte'] + df3['aporte']
df3['retirada'] = df3['vlr_resgate'] + df3['retirada']
df3.drop(columns=['vlr_aporte', 'vlr_resgate'], inplace=True)

In [9]:
#df2[df2['Papel'] == 'BCFF11'].sort_values(['ano', 'mes']).tail()
df3.sort_values(['Papel', 'ano', 'mes']).head(30)

,index,Papel,Qtd Disponivel,Cotacao,Financeiro,period,mes,ano,data_posicao,dividendo,aporte,retirada,rendimento_resgatado_x,rendimento_resgatado_y
82,82.00,AAPL34,16.00,72.02,"1,152.32",2021/1,1,2021,2021-01-29 00:00:00,0.00,"1,126.88",0.00,0.00,0.00
100,100.00,AAPL34,16.00,69.69,"1,115.04",2021/2,2,2021,2021-02-26 00:00:00,0.00,0.00,0.00,0.00,0.00
68,68.00,AMZO34,48.00,109.30,"5,246.40",2020/12,12,2020,2020-12-30 00:00:00,0.00,"5,011.20",0.00,0.00,0.00
83,83.00,AMZO34,48.00,112.00,"5,376.00",2021/1,1,2021,2021-01-29 00:00:00,0.00,0.00,0.00,0.00,0.00
101,101.00,AMZO34,48.00,110.22,"5,290.56",2021/2,2,2021,2021-02-26 00:00:00,0.00,0.00,0.00,0.00,0.00
560,560.00,AZ Quest Multi FIC FIM,"1,516.42",9.95,"15,094.10",2017/3,3,2017,2017-03-31 00:00:00,0.00,"30,000.00",0.00,0.00,0.00
556,556.00,AZ Quest Multi FIC FIM,"1,516.42",10.00,"15,160.90",2017/4,4,2017,2017-04-28 00:00:00,0.00,0.00,0.00,0.00,0.00
552,552.00,AZ Quest Multi FIC FIM,"1,516.42",9.73,"14,749.34",2017/5,5,2017,2017-05-31 00:00:00,0.00,0.00,0.00,0.00,0.00
548,548.00,AZ Quest Multi FIC FIM,"1,516.42",9.80,"14,853.49",2017/6,6,2017,2017-06-30 00:00:00,0.00,0.00,0.00,0.00,0.00
544,544.00,AZ Quest Multi FIC FIM,"1,516.42",10.15,"15,389.53",2017/7,7,2017,2017-07-31 00:00:00,0.00,0.00,0.00,0.00,0.00


In [10]:
df3[df3['Papel'] == 'BCFF11'].sort_values(['ano', 'mes']).tail()

,index,Papel,Qtd Disponivel,Cotacao,Financeiro,period,mes,ano,data_posicao,dividendo,aporte,retirada,rendimento_resgatado_x,rendimento_resgatado_y
270,270.00,BCFF11,354.00,87.25,"30,886.50",2020/10,10,2020,2020-10-30 00:00:00,24.30,"27,180.00",0.00,0.00,0.00
391,391.00,BCFF11,354.00,89.94,"31,838.76",2020/11,11,2020,2020-11-30 00:00:00,177.00,0.00,0.00,0.00,0.00
397,397.00,BCFF11,412.00,92.92,"38,283.04",2020/12,12,2020,2020-12-30 00:00:00,187.62,"5,138.80",0.00,0.00,0.00
404,404.00,BCFF11,492.00,92.00,"49,864.00",2021/1,1,2021,2021-01-29 00:00:00,218.36,"23,985.00",0.00,0.00,0.00
412,412.00,BCFF11,542.00,92.35,"50,053.70",2021/2,2,2021,2021-02-26 00:00:00,303.52,0.00,0.00,0.00,0.00


In [11]:
df3.groupby(['Papel', 'mes', 'ano']).agg(cont=('Papel', 'count')).reset_index().sort_values('cont', ascending=False).head()

,Papel,mes,ano,cont
0,AAPL34,1,2021,1
535,Polo Norte I FIC FIM,5,2019,1
513,PETR4,3,2020,1
514,PETR4,4,2020,1
515,PETR4,5,2020,1


In [12]:
extract.extract_fis.head()

,Nome,ano,mes,Vlr Aporte,Vlr Resgate,Vlr IR,Rendimento Resgatado
0,AZ Quest Multi FIC FIM,2017,3,"15,000.00",0.00,0.00,0.00
1,Azul Quantitativo FIM,2013,6,"5,000.00",0.00,0.00,0.00
2,Azul Quantitativo FIM,2013,8,"10,200.00",0.00,0.00,0.00
3,BNP Paribas Inflação FI RF,2015,1,"5,000.00",0.00,0.00,0.00
4,BNP Paribas Inflação FI RF,2015,2,"5,300.00",0.00,0.00,0.00


In [13]:
## Estou aqui!!! Falta dar o merge do extrato, começando com o de ações e fiis. 

df.sort_values(['Papel', 'ano', 'mes']).head()

,index,Papel,Qtd Disponivel,Cotacao,Financeiro,period,mes,ano,data_posicao,dividendo,aporte,retirada,rendimento_resgatado
82,82.00,AAPL34,16.00,72.02,"1,152.32",2021/1,1,2021,2021-01-29 00:00:00,0.00,"1,126.88",0.00,0.00
100,100.00,AAPL34,16.00,69.69,"1,115.04",2021/2,2,2021,2021-02-26 00:00:00,0.00,0.00,0.00,0.00
68,68.00,AMZO34,48.00,109.30,"5,246.40",2020/12,12,2020,2020-12-30 00:00:00,0.00,"5,011.20",0.00,0.00
83,83.00,AMZO34,48.00,112.00,"5,376.00",2021/1,1,2021,2021-01-29 00:00:00,0.00,0.00,0.00,0.00
101,101.00,AMZO34,48.00,110.22,"5,290.56",2021/2,2,2021,2021-02-26 00:00:00,0.00,0.00,0.00,0.00


In [14]:
df[df['Papel'] == 'BCFF11'].sort_values('data_posicao').tail()

,index,Papel,Qtd Disponivel,Cotacao,Financeiro,period,mes,ano,data_posicao,dividendo,aporte,retirada,rendimento_resgatado
270,270.00,BCFF11,354.00,87.25,"30,886.50",2020/10,10,2020,2020-10-30 00:00:00,24.30,"27,180.00",0.00,0.00
391,391.00,BCFF11,354.00,89.94,"31,838.76",2020/11,11,2020,2020-11-30 00:00:00,177.00,0.00,0.00,0.00
397,397.00,BCFF11,412.00,92.92,"38,283.04",2020/12,12,2020,2020-12-30 00:00:00,187.62,"5,138.80",0.00,0.00
404,404.00,BCFF11,492.00,92.00,"49,864.00",2021/1,1,2021,2021-01-29 00:00:00,218.36,"23,985.00",0.00,0.00
412,412.00,BCFF11,542.00,92.35,"50,053.70",2021/2,2,2021,2021-02-26 00:00:00,303.52,0.00,0.00,0.00


In [15]:
df_ext[df_ext['Papel'].str.contains('OGXP3')].sort_values('Data')

NameError: name 'df_ext' is not defined

In [9]:
extract.extrato_acoes.head()

,Papel,Categoria,Tipo,Qtde,Preco,Data,ano,mes
0,OGXP3,ACAO,COMPRA,500,9.91,2012-06-04,2012,6
1,OGXP3,ACAO,VENDA,500,1.85,2013-04-26,2013,4
6,MGLU3,ACAO,COMPRA,800,34.50,2019-09-12,2019,9
9,JHSF3,ACAO,COMPRA,1600,8.09,2020-01-31,2020,1
10,BPAC11,ACAO,COMPRA,200,35.95,2020-03-13,2020,3


In [10]:
extract.extrato_fiis.head()

,Papel,Categoria,Tipo,Qtde,Preco,Data,ano,mes
2,CNES11,FII,COMPRA,11,80.47,2014-04-30,2014,4
3,TBOF11,FII,COMPRA,100,52.00,2014-07-31,2014,7
4,RNGO11,FII,COMPRA,49,83.89,2014-07-31,2014,7
5,TBOF11,FII,VENDA,100,51.99,2015-09-30,2015,9
7,OULG11,FII,COMPRA,50,113.55,2020-01-20,2020,1


In [11]:
position.fiis.head()

,Papel,Qtd Disponivel,Qtd Projetada,Qtd Dia,Qtde Total,Ult Cotacao,Financeiro,period,mes,ano,data_posicao
0,BCFF11,8,0,0,8.00,85.87,686.96,2019/5,5,2019,2019-05-31
1,CNES11,11,0,0,11.00,76.00,836.00,2019/5,5,2019,2019-05-31
2,RNGO11,49,0,0,49.00,84.30,"4,130.70",2019/5,5,2019,2019-05-31
3,BCFF11,8,0,0,8.00,85.13,681.04,2020/4,4,2020,2020-04-30
4,CNES11,11,0,0,11.00,70.00,770.00,2020/4,4,2020,2020-04-30


In [12]:
position.fis[(position.fis['Valor Bruto'] != position.fis['Total Bruto']) & (~position.fis['Total Bruto'].isnull())].head(100)

,Nome,Data,Qtd Cotas,Valor Cota,Valor Bruto,IR,IOF,Valor Liquido,Aplicacao Pendente,Total Bruto,period,mes,ano,data_posicao


In [13]:
## Estou aqui!! Juntei os datasets de posição, falta agora os de dividendos. 

#print(position.stocks.columns)
#print(position.fiis.columns)
#print(position.fis.columns)

df1 = position.stocks[['Papel', 'Qtd Disponivel',
       'Cotacao', 'Financeiro', 'period', 'mes', 'ano',
       'data_posicao']]

df2 = position.fiis[['Papel', 'Qtd Disponivel', 
            'Ult Cotacao', 'Financeiro', 'period', 'mes', 'ano', 
            'data_posicao']]\
           .rename(columns={'Ult Cotacao': 'Cotacao'})

df2['Financeiro'] = np.where(df2['Financeiro'] == 0, 
        df2['Qtd Disponivel'] * df2['Cotacao'], 
        df2['Financeiro']) # trata os valores vazios de Financeiro. 

# ToDo: Depois acrescentar esses campos: 'IR', 'IOF', 'Valor Liquido'
df3 = position.fis[['Nome', 'Qtd Cotas', 'Valor Cota', 'Valor Bruto', 
                    'period', 'mes',
                    'ano', 'data_posicao']]\
            .rename(columns={'Nome': 'Papel',
                            'Qtd Cotas': 'Qtd Disponivel', 
                            'Valor Cota': 'Cotacao', 
                            'Valor Bruto': 'Financeiro'})                            

df4 = position.stocks_profits\
        .groupby(['Papel', 'data_posicao'])\
            .agg(Valor=('Valor', 'sum'))\
        .reset_index()

df5 = extract.fiis_profits\
            .groupby(['Descricao', 'ano', 'mes'])\
                .agg(Valor=('Valor', 'sum'))\
            .reset_index()\
            .rename(columns={'Descricao': 'Papel'})

df_result = df1.append(df2, ignore_index=True).append(df3, ignore_index=True)   


df_result = df_result.merge(df4, 
                        how='outer', 
                        left_on=['Papel', 'data_posicao'], 
                        right_on=['Papel', 'data_posicao'])\
                    .fillna(0)
                    
df_result = df_result.merge(df5, 
                        how='left', # existem dividendos de periodos sem posicao ex.: BCFF11
                        left_on=['Papel', 'ano', 'mes'], 
                        right_on=['Papel', 'ano', 'mes'])\
                    .fillna(0)
df_result['dividendo'] = df_result['Valor_x'] + df_result['Valor_y']
df_result.drop(columns=['Valor_x', 'Valor_y'], inplace=True)


df_result.sort_values(by=['data_posicao', 'Papel']).tail()

,Papel,Qtd Disponivel,Cotacao,Financeiro,period,mes,ano,data_posicao,dividendo
417,RBRF11,119.00,94.46,"11,240.74",2021/2,2,2021,2021-02-26,5.54
418,RNGO11,49.00,64.49,"3,160.01",2021/2,2,2021,2021-02-26,19.11
116,TSLA34,50.00,118.00,"5,900.00",2021/2,2,2021,2021-02-26,0.00
117,WEGE3,300.00,77.70,"23,310.00",2021/2,2,2021,2021-02-26,21.40
419,XPLG11,79.00,120.50,"9,519.50",2021/2,2,2021,2021-02-26,31.46


In [14]:
df2[df2['Papel'] == 'BCFF11'].sort_values('data_posicao').head(10)

,Papel,Qtd Disponivel,Cotacao,Financeiro,period,mes,ano,data_posicao
258,BCFF11,8,81.10,648.80,2014/5,5,2014,2014-05-30
254,BCFF11,8,81.06,648.48,2014/6,6,2014,2014-06-30
250,BCFF11,8,72.50,580.00,2014/7,7,2014,2014-07-31
246,BCFF11,8,72.90,583.20,2014/8,8,2014,2014-08-29
242,BCFF11,8,70.00,560.00,2014/9,9,2014,2014-09-30
238,BCFF11,8,66.80,534.40,2014/10,10,2014,2014-10-31
228,BCFF11,8,63.49,507.92,2014/11,11,2014,2014-11-28
224,BCFF11,8,67.00,536.00,2014/12,12,2014,2014-12-30
220,BCFF11,8,66.84,534.72,2015/1,1,2015,2015-01-30
216,BCFF11,8,65.29,522.32,2015/2,2,2015,2015-02-27


In [15]:
df5[df5['Valor'] == 0].head()

,Papel,ano,mes,Valor


In [16]:
print(df2.info())
print(df5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Papel           302 non-null    object        
 1   Qtd Disponivel  302 non-null    int64         
 2   Cotacao         302 non-null    float64       
 3   Financeiro      302 non-null    float64       
 4   period          302 non-null    object        
 5   mes             302 non-null    int64         
 6   ano             302 non-null    int64         
 7   data_posicao    302 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 19.0+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Papel   395 non-null    object 
 1   ano     395 non-null    int64  
 2   mes     395 non-null    int64  


In [17]:
df_result[df_result['Valor_y'] > 0].head()

KeyError: 'Valor_y'

In [18]:
df5 = extract.fiis_profits\
            .groupby(['Descricao', 'Mov'])\
                .agg(Valor=('Valor', 'sum'))\
            .reset_index()\
            .rename(columns={'Descricao': 'Papel', 
                            'Mov': 'Data'})

df2 = position.stocks_profits\
        .groupby(['Papel', 'data_posicao'])\
            .agg(Valor=('Valor', 'sum'))\
        .reset_index().sort_values('Papel')\
        .rename(columns={'data_posicao': 'Data'})                            

df3 = df1.append(df2, ignore_index=True)

In [56]:
df3.head()

,Papel,Qtd Disponivel,Cotacao,Financeiro,period,mes,ano,data_posicao
0,XP Long Short FIC FIM,"6,914.97",2.84,"19,638.20",2019/5,5,2019,2019-05-31
1,AZ Quest Multi FIC FIM,"1,476.76",11.85,"17,496.02",2019/5,5,2019,2019-05-31
2,Mauá Macro FIC FIM,37.37,706.87,"26,413.30",2019/5,5,2019,2019-05-31
3,Bahia AM Maraú FIC de FIM,"33,993.07",2.45,"83,382.14",2019/5,5,2019,2019-05-31
4,XP Macro FIM,"7,128.20",1.45,"10,353.74",2019/5,5,2019,2019-05-31


In [57]:
extract.fiis_profits.head()

,Liq,Mov,Descricao,Valor,Saldo,ano,mes
4,2020-05-28,2020-05-28,CNES11,1.81,"1,478.00",2020,5
5,2020-05-22,2020-05-22,OULG11,32.80,"1,476.19",2020,5
6,2020-05-19,2020-05-19,IBFF11,11.91,"1,443.39",2020,5
7,2020-05-15,2020-05-15,RNGO11,20.58,"1,431.48",2020,5
8,2020-05-15,2020-05-15,BCFF11,3.20,"1,410.90",2020,5
